# Day 8

- The first portion of the input is directions
- The second portion is mappings between points
- count the number of iterations it takes to get to `ZZZ` starting at `AAA`

In [11]:
from itertools import cycle

from tqdm import tqdm

from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig, markdown
)

In [12]:
def extract_options(option_string: str) -> tuple[str, str]:
    """does the final step parsing the mapping options"""
    return option_string[1:-1].split(', ')

parser = ParseConfig('\n\n', [
    ParseConfig('', str),
    ParseConfig('\n', ParseConfig(' = ', [str, extract_options]))
])

directions, mappings = parse_from_file('puzzle_input\\day_8.txt', parser)

print(directions[:3])
print(mappings[:3])

['L', 'L', 'L']
[['MQF', ['DDG', 'LSH']], ['QJP', ['PCT', 'XKJ']], ['JXF', ['PMG', 'NBN']]]


In [13]:
mapping_dict = {point: options for point, options in mappings}
direction_cycle = cycle([direction == 'R' for direction in directions])

In [14]:
location = 'AAA'
iterations = 0
for direction in tqdm(direction_cycle):
    iterations += 1
    location = mapping_dict[location][direction]
    if location == 'ZZZ':
        break

12082it [00:00, 734205.26it/s]


In [15]:
markdown(
    '### Solution',
    f'The number of iterations it will take to get to ZZZ is: {iterations}'
)

### Solution
The number of iterations it will take to get to ZZZ is: 12083